# Read in the parquet data tables via dask.DataFrames and create MySQL tables

NOTE: this might put weird values into the sql table. I wouldn't trust.

In [1]:
import sqlalchemy as sa
import pymysql
import dask.dataframe as dd
import pandas as pd
from dask.diagnostics import ProgressBar
import replicaEVSE.sql_wrapper_functions as sql
pbar = ProgressBar()
pbar.register()

pd.set_option('display.max_columns', None)

with open('../src/metadata/MySQLpwd.txt') as f:
    pw = f.readlines()
pw = pw[0]

# define where the data is
popparquet = 'northwest_2021_Q4_population.parquet'
tripsatparquet = 'northwest_2021_Q4_saturday_trip.parquet'
tripthuparquet = 'northwest_2021_Q4_thursday_trip.parquet'
datapath = '../../data/'

# read in the data
pop_df = dd.read_parquet(datapath + popparquet)
trip_sat_df = dd.read_parquet(datapath + tripsatparquet)
trip_thu_df = dd.read_parquet(datapath + tripthuparquet)

# add in a weekday column to the trips data
trip_sat_df['weekday'] = 'saturday'
trip_thu_df['weekday'] = 'thursday'

# sort the pop data. ~ 5 minutes
sorted_pop_df = pop_df.sort_values(by='person_id', ascending=True)

# these two tables will get appended so no need to sort
# sorted_trip_sat_df = trip_sat_df.sort_values(by='person_id', ascending=True)
# sorted_trip_thu_df = trip_thu_df.sort_values(by='person_id', ascending=True)


# these are the general parameters for the connection
# make sure you defined your password in the metadata folder
user = 'root'
server = 'localhost'
database = 'replica'
to_sql_uri = f'mysql+pymysql://{user}:{pw}@{server}/{database}'   




[########################################] | 100% Completed | 34.02 s


## Here is the step where we add the data to the MySQL server


In [ ]:
# this will create the database if it doesn't exist
# if you want to append to an existing database, change if_exists to 'append'
sorted_pop_df.to_sql('population', uri=to_sql_uri, if_exists='replace', index=False)

SQL didnt like the timedelta dtype so converting it. 

In [22]:
trip_sat_df
trip_sat_df['start_time'].dtype

dtype('<m8[ns]')

In [35]:
trip_thu_df = dd.read_parquet(datapath + tripthuparquet)
trip_thu_df['start_time'].dtype

dtype('<m8[ns]')

In [36]:
trip_thu_df = dd.read_parquet(datapath + tripthuparquet)

trip_sat_df['start_time'] = trip_sat_df['start_time'].values.astype('datetime64[ns]')
trip_sat_df['end_time'] = trip_sat_df['end_time'].values.astype('datetime64[ns]')

trip_thu_df['start_time'] = trip_thu_df['start_time'].values.astype('datetime64[ns]')
trip_thu_df['end_time'] = trip_thu_df['end_time'].values.astype('datetime64[ns]')

In [37]:
trip_thu_df['start_time'].dtype

dtype('<M8[ns]')

In [38]:
trip_thu_df.dtypes

activity_id                            object
person_id                              object
mode                                   object
travel_purpose                         object
previous_activity_type                 object
start_time                     datetime64[ns]
end_time                       datetime64[ns]
distance_miles                        float64
vehicle_type                           object
origin_bgrp                            object
origin_bgrp_lat                       float64
origin_bgrp_lng                       float64
destination_bgrp                       object
destination_bgrp_lat                  float64
destination_bgrp_lng                  float64
origin_land_use_l1                     object
origin_land_use_l2                     object
origin_building_use_l1                 object
origin_building_use_l2                 object
destination_land_use_l1                object
destination_land_use_l2                object
destination_building_use_l1       

In [25]:
trip_sat_df.to_sql('trips', uri=to_sql_uri, if_exists='replace', index=False)

[########################################] | 100% Completed | 38m 42s
[########################################] | 100% Completed | 38m 42s


In [39]:
trip_thu_df.to_sql('trips', uri=to_sql_uri, if_exists='append', index=False)

[########################################] | 100% Completed | 42m 49s
[########################################] | 100% Completed | 42m 49s


# create tables in our replica db to add values to.

In [4]:
create_charges_table="""
CREATE TABLE charges (
  charge_id VARCHAR(24) PRIMARY KEY,
  activity_id VARCHAR(20),
  simulation_id VARCHAR(4),
  charger_power_kW DECIMAL(5,1),
  charge_energy_used_kWh DECIMAL(4,1),
  charge_opportunity_remaining_kWh DECIMAL(5,1)
  );
"""
create_load_table="""
CREATE TABLE loads (
  load_segment_id VARCHAR(26) PRIMARY KEY,
  charge_id VARCHAR(24),
  window_start_time TIME,
  window_end_time TIME,
  load_kW DECIMAL(5,1)
  );
"""
create_simulation_table="""
CREATE TABLE simulation_runs (
  simulation_id VARCHAR(4) PRIMARY KEY,
  simulation_date DATETIME,
  simulation_attributes VARCHAR(200)
  );
"""

In [5]:
import replicaEVSE.sql_wrapper_functions as sql
connection_replica = sql.create_db_connection("localhost", "root", pw, 'replica') # Connect to the Database
sql.execute_query(connection_replica, 'DROP TABLE IF EXISTS loads;')
sql.execute_query(connection_replica, 'DROP TABLE IF EXISTS charges;')
sql.execute_query(connection_replica, 'DROP TABLE IF EXISTS simulation_runs;')
sql.execute_query(connection_replica, create_charges_table) # Execute our defined query
sql.execute_query(connection_replica, create_load_table) # Execute our defined query
sql.execute_query(connection_replica, create_simulation_table) # Execute our defined query

MySQL Database connection successful
